In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY

import mysql.connector
from mysql.connector import errorcode
from ittutils import ittconnection, get_resampled, get_raw_price, get_raw_volume, classification_dataset_from_ts

/Users/alex/anaconda3/envs/py3.6_ITT/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Create sql for modifieng signals

In [5]:

# change in the real db the sign of trend
update_1_query = (
    " UPDATE indicator_annpriceclassification SET ai_model = 'PRICE_PREDICT' WHERE ai_model is Null "
)


# Check staging db before update

In [6]:
db_connection = ittconnection('stage')

query = (" SELECT * FROM indicator_annpriceclassification WHERE ai_model is Null LIMIT 100 ")
indicators_df = pd.read_sql(query, con=db_connection)
indicators_df.tail(6)

,id,source,counter_currency,transaction_currency,timestamp,resample_period,predicted_ahead_for,probability_same,probability_up,probability_down,ann_model_id,ai_model
94,95,0,0,DCR,1.522843e+09,240,90,0.382280,0.301476,0.316244,1,None
95,96,0,0,LBC,1.522843e+09,60,90,0.468817,0.239539,0.291644,1,None
96,97,0,0,FLO,1.522843e+09,240,90,0.439256,0.242283,0.318461,1,None
97,98,0,0,GNO,1.522843e+09,60,90,0.434065,0.277209,0.288727,1,None
98,99,0,0,POT,1.522843e+09,240,90,0.435936,0.259266,0.304798,1,None
99,100,0,0,DCR,1.522843e+09,60,90,0.382280,0.301476,0.316244,1,None


# Update Staging DB

In [7]:
db_connection = ittconnection('stage')
cursor = db_connection.cursor()
cursor.execute(update_1_query)

db_connection.commit()
cursor.close()
db_connection.close()

## check DB after update

In [9]:
db_connection = ittconnection('stage')
query = (" SELECT * FROM indicator_annpriceclassification LIMIT 100 ")
new_indicator_df = pd.read_sql(query, con=db_connection)
new_indicator_df.tail(5)


,id,source,counter_currency,transaction_currency,timestamp,resample_period,predicted_ahead_for,probability_same,probability_up,probability_down,ann_model_id,ai_model
95,1651385,1,0,TX,1.536174e+09,60,480,0.187534,0.392727,0.419739,2,PRICE_PREDICT
96,1651387,1,0,BLOCK,1.536174e+09,60,480,0.296820,0.329226,0.373954,2,PRICE_PREDICT
97,1651389,1,0,XEL,1.536174e+09,60,480,0.280829,0.287305,0.431865,2,PRICE_PREDICT
98,1651392,1,0,GO,1.536174e+09,60,480,0.262924,0.261244,0.475832,2,PRICE_PREDICT
99,1651394,1,0,XDN,1.536174e+09,60,480,0.348453,0.255839,0.395707,2,PRICE_PREDICT
